In [21]:
import akshare as ak
import pandas as pd
stock_zh_a_spot_df = ak.stock_zh_a_spot_em()

In [22]:
filter_data = stock_zh_a_spot_df[['代码', '名称', '最新价', '涨跌幅','市盈率-动态', '市净率', '总市值']]

selected_stocks = pd.DataFrame(filter_data)

# 保存到Excel文件
selected_stocks.to_excel('base_stocks.xlsx', index=False)

print("Stocks with market cap less than 200 billion exported to Excel successfully.")

Stocks with market cap less than 200 billion exported to Excel successfully.


In [23]:
import re
# & ~stock_zh_a_spot_df['名称'].str.contains('ST') & ~stock_zh_a_spot_df['名称'].str.contains('\*ST')]
filtered_stocks = {'name': [], 'code': [], 'market_cap': [],'price': []}
for index,stock_item in enumerate(stock_zh_a_spot_df['总市值']):
    stock_name = stock_zh_a_spot_df['名称'][index]
    market_cap_item = stock_item/100000000
    stock_code = stock_zh_a_spot_df['代码'][index]
    if not re.search(r'st|\*st', stock_name, re.IGNORECASE) and  market_cap_item < 100 and re.search(r'600|601|603|000|001|002', stock_code[0:3]):
            filtered_stocks['name'].append(stock_zh_a_spot_df['名称'][index])
            filtered_stocks['code'].append(stock_code)
            filtered_stocks['market_cap'].append(market_cap_item)
            filtered_stocks['price'].append(stock_zh_a_spot_df['最新价'][index])

print(len(filtered_stocks['name']))
print(len(filtered_stocks['code']))


1976
1976


In [24]:

# [{
# 'name': '中航光电', 
# 'code': '600184', 
# 'market _cap': 0.0, 
# 'limit_up_num': '10', 
# 'is_T_limit_up: true/false',
# 'three_day_amplitude': '10.0'
# 'five_day_amplitude': '10.0'
# }]
limit_up_stocks = []
endDate = '20240124'
startDate = '20240123'
sliceLen = len(filtered_stocks['code'])
for s_index,stock_code in enumerate(filtered_stocks['code'][0:sliceLen]):
    stock_hist = ak.stock_zh_a_hist(symbol=stock_code, start_date=startDate, end_date=endDate)
    # print(index,stock_code,stock_hist)
    if not stock_hist.empty:
        list = stock_hist.iterrows()
        limit_up_data = {}
        list_item = {
            'name': filtered_stocks['name'][s_index], 
            'code': filtered_stocks['code'][s_index], 
            'price': filtered_stocks['price'][s_index],
            'market_cap': filtered_stocks['market_cap'][s_index], 
            'limit_up_num': 0, 
            'limit_up_type': '',
            'is_T_limit_up': False, 
            'limit_up_array': [],
            'tody_rise_percent': 0,
            'three_day_amplitude': 0, 
            'five_day_amplitude': 0
        }
        limit_up_num = 0
        for index, row in list:
            current_row = row
            current_index = index
            # 获取下一行的索引
            next_index = stock_hist.index.get_loc(index) + 1

            # 检查是否到达最后一行
            if next_index < len(stock_hist):
                next_row = stock_hist.loc[next_index]
                stock_rise_percent = (next_row['收盘'] - current_row['收盘']) / current_row['收盘'] * 100
                if stock_rise_percent > 9.9 and next_row['收盘'] == next_row['最高']:
                    print(index, stock_code,filtered_stocks['name'][s_index])
                    type = ''
                    limit_up_num = limit_up_num + 1
                    current_item = {'index': next_index, 'rise_percent': stock_rise_percent, 'type': ''}
                    prev_index = 0
                    prev_item = {}
                    if limit_up_num == 2:
                        prev_item = limit_up_data[limit_up_num - 1]
                        prev_index = prev_item['index']
                    list_item['tody_rise_percent'] = stock_rise_percent
                    if next_row['收盘'] == next_row['开盘'] and next_row['收盘'] > current_row['最低']:
                        type = 'T1'
                        if prev_index == next_index - 1:
                            list_item['is_T_limit_up'] = True
                            
                        print('T1',index,stock_code, stock_rise_percent)
                    elif next_row['收盘'] >= next_row['开盘'] and current_row['开盘'] > current_row['最低']:
                        type = 'T2'
                        print('T2',index,stock_code, stock_rise_percent)
                    else:
                        type = 'T3'
                        print('T3',index,stock_code, stock_rise_percent)
                    current_item['type'] = type
                    limit_up_data[limit_up_num] = current_item
                    list_item['limit_up_num'] = limit_up_num
                    list_item['limit_up_type'] = type
                    list_item['limit_up_array'].append(limit_up_data)
                    print('list_item',list_item)
                    limit_up_stocks.append(list_item)
                # else:
                    
                    # if limit_up_num == 2 and list_item['is_T_limit_up'] and and stock_rise_percent < -5:




0 002172 澳洋健康
T2 0 002172 10.144927536231874
list_item {'name': '澳洋健康', 'code': '002172', 'price': 3.8, 'market_cap': 29.09782968, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_percent': 10.144927536231874, 'type': 'T2'}}], 'tody_rise_percent': 10.144927536231874, 'three_day_amplitude': 0, 'five_day_amplitude': 0}
0 600626 申达股份
T2 0 600626 10.126582278481006
list_item {'name': '申达股份', 'code': '600626', 'price': 3.48, 'market_cap': 45.96191441, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_percent': 10.126582278481006, 'type': 'T2'}}], 'tody_rise_percent': 10.126582278481006, 'three_day_amplitude': 0, 'five_day_amplitude': 0}
0 600191 华资实业
T2 0 600191 10.088495575221227
list_item {'name': '华资实业', 'code': '600191', 'price': 6.22, 'market_cap': 30.1627704, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_perc

In [25]:
print('limit_up_stocks',limit_up_stocks)
selected_stocks = pd.DataFrame(limit_up_stocks)
# 构建导出文件名，包含每日日期
export_filename = f"{startDate}_{endDate}.xlsx"
# 保存到Excel文件
selected_stocks.to_excel(export_filename, index=False)

print("excel has been exported successfully.")


limit_up_stocks [{'name': '澳洋健康', 'code': '002172', 'price': 3.8, 'market_cap': 29.09782968, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_percent': 10.144927536231874, 'type': 'T2'}}], 'tody_rise_percent': 10.144927536231874, 'three_day_amplitude': 0, 'five_day_amplitude': 0}, {'name': '申达股份', 'code': '600626', 'price': 3.48, 'market_cap': 45.96191441, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_percent': 10.126582278481006, 'type': 'T2'}}], 'tody_rise_percent': 10.126582278481006, 'three_day_amplitude': 0, 'five_day_amplitude': 0}, {'name': '华资实业', 'code': '600191', 'price': 6.22, 'market_cap': 30.1627704, 'limit_up_num': 1, 'limit_up_type': 'T2', 'is_T_limit_up': False, 'limit_up_array': [{1: {'index': 1, 'rise_percent': 10.088495575221227, 'type': 'T2'}}], 'tody_rise_percent': 10.088495575221227, 'three_day_amplitude': 0, 'five_day_amplitude': 0}, {'name': '